# Utility

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import tensorflow as tf
from keras import backend as K
from sklearn.utils import class_weight as cw

class utility:

    def read_CSV(self, filename):
        df = pd.read_csv(filename, encoding= 'unicode_escape')
        return df

    def get_text_label(self, df):
        texts = []  # list of text samples
        labels = []  # list of label ids
        for index, row in df.iterrows():
            if isinstance(row['text'], float):
                texts.append(str(row['text']))
            else:
                texts.append(row['text'])

            labels.append(row['sentiment'])

        return texts, labels

    def tokenize_texts(self, texts):
        tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
        tokenizer.fit_on_texts(texts)

        return tokenizer

    def padding_texts(self, texts, maxlen):

        texts = tf.keras.preprocessing.sequence.pad_sequences(texts, padding='post', maxlen=maxlen)

        return texts

    def get_metric(self, y_true, y_pred):
        accuracyScore = accuracy_score(y_true, y_pred)

        # binary: Only report results for the class specified by pos_label. This is applicable only if targets (y_{true,pred}) are binary.
        precisionScoreBinary = precision_score(y_true, y_pred, average='binary')
        recallScoreBinary = recall_score(y_true, y_pred, average='binary')
        f1ScoreBinary = f1_score(y_true, y_pred, average='binary')

        return accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary

    def print_metric(self, accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary):
        print("Accuracy: " + str(accuracyScore))
        print("Precision: " + str(precisionScoreBinary))
        print("Recall: " + str(recallScoreBinary))
        print("F1-Score: " + str(f1ScoreBinary))
        print(str(accuracyScore) + "," + str(precisionScoreBinary) + "," + str(recallScoreBinary) + "," + str(
            f1ScoreBinary))

    def get_testing_metric(self, y_test, y_pred):
        # metric for Testing Data
        # print("Testing Data")
        accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = self.get_metric(y_test, y_pred)
        # print()

        return accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary

    def write_df_csv(self, df, out_path):
        df.to_csv(out_path, index=False)

    def create_embedding_matrix(self, filepath, word_index, embedding_dim):
        vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
        embedding_matrix = np.zeros((vocab_size, embedding_dim))

        with open(filepath, encoding="utf8") as f:
            for line in f:
                word, *vector = line.split()
                if word in word_index:
                    idx = word_index[word]
                    embedding_matrix[idx] = np.array(
                        vector, dtype=np.float32)[:embedding_dim]

        return embedding_matrix

    def get_max_length_of_sentences(self, texts):
        maxlength = 0
        for text in texts:
            if (len(text.split()) > maxlength):
                maxlength = len(text.split())

        return maxlength

    def get_training_val_data(self, textsTraining, labelsTraining, train_index, val_index, glovePath):
        textsTraining, textsTesting = np.asarray(textsTraining)[train_index], np.asarray(textsTraining)[val_index]
        y_train, y_val = np.asarray(labelsTraining)[train_index], np.asarray(labelsTraining)[val_index]

        # Tokenize words
        tokenizer = self.tokenize_texts(textsTraining)
        X_train = tokenizer.texts_to_sequences(textsTraining)
        X_val = tokenizer.texts_to_sequences(textsTesting)

        # Adding 1 because of reserved 0 index
        vocab_size = len(tokenizer.word_index) + 1

        # get maxlen
        maxlen = self.get_max_length_of_sentences(textsTraining)

        # Pad sequences with zeros
        X_train = self.padding_texts(X_train, maxlen)
        X_val = self.padding_texts(X_val, maxlen)

        embedding_matrix = []
        embedding_matrix.append(self.create_embedding_matrix(glovePath[0], tokenizer.word_index, 50))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[1], tokenizer.word_index, 100))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[2], tokenizer.word_index, 200))
        embedding_matrix.append(self.create_embedding_matrix(glovePath[3], tokenizer.word_index, 300))

        return X_train, X_val, y_train, y_val, vocab_size, maxlen, embedding_matrix

    def Average(self, list):
        return sum(list) / len(list)

    def recall_m(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision_m(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def f1_m(self, y_true, y_pred):
        precision = self.precision_m(y_true, y_pred)
        recall = self.recall_m(y_true, y_pred)
        return 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    def get_weight(self, y):
        class_weight_current =  cw.compute_class_weight('balanced', np.unique(y), y)
        return class_weight_current

Using TensorFlow backend.


# CNN

In [2]:
import tensorflow as tf

class CNN:

    def cnn_model(self, vocab_size, maxlen, embedding_matrix, indiv, path):
        model = tf.keras.models.Sequential()
        conv_idx = dense_idx = dropout_idx = maxpooling_idx = 0
        for layer in path:
            if layer == 'embedding_layer':
                model.add(
                    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=indiv['output_dim'],
                                     weights=[embedding_matrix], input_length=maxlen, trainable=True))
            elif layer == 'convolutional_layer':
                conv_idx += 1
                model.add(tf.keras.layers.Conv1D(indiv['num_filters' + str(conv_idx)], indiv['kernel_size' + str(conv_idx)],
                                        kernel_initializer=indiv['conv_init_mode' + str(conv_idx)],
                                        activation=indiv['conv_activation_func' + str(conv_idx)],
                                        kernel_constraint=tf.keras.constraints.max_norm(indiv['conv_weight_constraint' + str(conv_idx)]),
                                        data_format='channels_first'))
            elif layer == 'dense_layer':
                dense_idx += 1
                model.add(tf.keras.layers.Dense(indiv['neurons' + str(dense_idx)],
                                       kernel_initializer=indiv['dense_init_mode' + str(dense_idx)],
                                       activation=indiv['dense_activation_func' + str(dense_idx)],
                                       kernel_constraint=tf.keras.constraints.max_norm(indiv['dense_weight_constraint' + str(dense_idx)])))
            elif layer == 'dropout_layer':
                dropout_idx += 1
                model.add(tf.keras.layers.Dropout(indiv['dropout_rate' + str(dropout_idx)]))
            elif layer == 'maxpooling_layer':
                maxpooling_idx += 1
                model.add(tf.keras.layers.MaxPooling1D(indiv['pool_size' + str(maxpooling_idx)]))
            elif layer == 'global_maxpooling_layer':
                model.add(tf.keras.layers.GlobalMaxPooling1D())
            elif layer == 'output_layer':
                model.add(tf.keras.layers.Dense(1, kernel_initializer=indiv['output_init_mode'], activation='sigmoid'))

        if indiv['optimizer'] == 'sgd':
            opt = tf.keras.optimizers.SGD(lr=indiv['learning_rate'], momentum=indiv['momentum'], decay=0.0,
                                 nesterov=False)
        elif indiv['optimizer'] == 'rmsprop':
            opt = tf.keras.optimizers.RMSprop(lr=indiv['learning_rate'], rho=0.9, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adagrad':
            opt = tf.keras.optimizers.Adagrad(lr=indiv['learning_rate'], epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adadelta':
            opt = tf.keras.optimizers.Adadelta(lr=indiv['learning_rate'], rho=0.95, epsilon=None, decay=0.0)
        elif indiv['optimizer'] == 'adam':
            opt = tf.keras.optimizers.Adam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                  decay=0.0, amsgrad=False)
        elif indiv['optimizer'] == 'adamax':
            opt = tf.keras.optimizers.Adamax(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                    decay=0.0)
        elif indiv['optimizer'] == 'nadam':
            opt = tf.keras.optimizers.Nadam(lr=indiv['learning_rate'], beta_1=0.9, beta_2=0.999, epsilon=None,
                                   schedule_decay=0.004)

        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

        return model

# Path

In [3]:
training_path = 'train.csv'
val_path = 'validation.csv'
root_path = '/lab/dbms/fatyanosa'
datasetPath = '{}/Dataset/Twitter US Airline Sentiment/'.format(root_path)
resultsPath = '{}/Server1/Twitter US Airline Sentiment/Paper DGGA-CNN/Results/'.format(root_path)
archPath = '{}/Server1/Twitter US Airline Sentiment/Paper DGGA-CNN/Architecture/'.format(root_path)
testing_name = 'GridSearch'
glovePath = ['{}/Glove/glove.6B.50d.txt'.format(root_path),
             '{}/Glove/glove.6B.100d.txt'.format(root_path),
             '{}/Glove/glove.6B.200d.txt'.format(root_path),
             '{}/Glove/glove.6B.300d.txt'.format(root_path)]

# Run

In [4]:
import time
import os
import csv
import gc
from sklearn.utils import class_weight as cw

# Optimized hyperparameters
epochs = [80]
batch_size = [256]#[32, 64, 128, 256]
optimizer = ['adam', 'nadam', 'adamax', 'sgd', 'rmsprop']
learning_rate = [1e-4, 1e-3]
output_dim = [100, 200, 300]
conv_activation_func1 = ['sigmoid', 'tanh', 'relu', 'elu', 'selu']

combination = [[h, i, j, k, l, m] for h in epochs
       for i in batch_size
       for j in optimizer
       for k in learning_rate
       for l in output_dim
       for m in conv_activation_func1]

# Fixed hyperparameters
hyperparameters = {}
hyperparameters['momentum'] = 0.9
hyperparameters['num_filters1'] = 64
hyperparameters['kernel_size1'] = 3
hyperparameters['conv_init_mode1'] = 'glorot_uniform'
hyperparameters['conv_weight_constraint1'] = 3
hyperparameters['dropout_rate1'] = 0.2
hyperparameters['neurons1'] = 1
hyperparameters['dense_activation_func1'] = 'relu'
hyperparameters['dense_init_mode1'] = 'glorot_uniform'
hyperparameters['dense_weight_constraint1'] = 3
hyperparameters['output_init_mode'] = 'glorot_uniform'
hyperparameters['layers'] = ['embedding_layer', 'convolutional_layer', 
                             'global_maxpooling_layer', 'dense_layer',
                              'dropout_layer', 'output_layer']

# object class
util = utility()
cnn = CNN()

# Read data
dfTraining = util.read_CSV("{}{}".format(datasetPath, training_path))
dfVal = util.read_CSV("{}{}".format(datasetPath, val_path))

# get texts and labels
textsTraining, y_train = util.get_text_label(dfTraining)
textsTesting, y_test = util.get_text_label(dfVal)

# Tokenize words
tokenizer = util.tokenize_texts(textsTraining)
X_train = tokenizer.texts_to_sequences(textsTraining)
X_test = tokenizer.texts_to_sequences(textsTesting)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# get maxlen
maxlen = util.get_max_length_of_sentences(textsTraining)

# Pad sequences with zeros
X_train = util.padding_texts(X_train, maxlen)
X_test = util.padding_texts(X_test, maxlen)


# Create Testing Results
f = open(resultsPath + testing_name + ".csv", "a+")
f.write("i,epochs,batch_size,optimizer,learning_rate,output_dim,conv_activation_func1,accuracy,precision,recall,f1Score,time\n")
f.close()

for i in range(len(combination)):
    print(str(i + 1))
    hyperparameters['epochs'] = combination[i][0]
    hyperparameters['batch_size'] = combination[i][1]
    hyperparameters['optimizer'] = combination[i][2]
    hyperparameters['learning_rate'] = combination[i][3]
    hyperparameters['output_dim'] = combination[i][4]
    hyperparameters['conv_activation_func1'] = combination[i][5]

    if int(hyperparameters['output_dim']) == 50:
        glove = glovePath[0]
    elif int(hyperparameters['output_dim']) == 100:
        glove = glovePath[1]
    elif int(hyperparameters['output_dim']) == 200:
        glove = glovePath[2]
    elif int(hyperparameters['output_dim']) == 300:
        glove = glovePath[3]

    embedding_matrix = util.create_embedding_matrix(glove, tokenizer.word_index, int(hyperparameters['output_dim']))

    then = time.time()
    # grid search
    model = cnn.cnn_model(vocab_size, maxlen, embedding_matrix, hyperparameters, hyperparameters['layers'])

    #early stopping
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', verbose=False, patience=10)]

    #save the best model
    callbacks += [tf.keras.callbacks.ModelCheckpoint(archPath + testing_name + str(i + 1) + ".h5", monitor='val_acc', mode='max', verbose=False, save_best_only=True)]
    
#     model.summary()
    y_train = np.uint8(y_train)
    y_test = np.uint8(y_test)
    model.fit(X_train, y_train, epochs=hyperparameters['epochs'], verbose=False, validation_data=(X_test, y_test),
              batch_size=hyperparameters['batch_size'], callbacks=callbacks, class_weight=util.get_weight(y_train))
    
    # save model to a file
    # model.save(archPath + testing_name + str(i + 1) + ".h5")

    # load the saved model
    saved_model = tf.keras.models.load_model(archPath + testing_name + str(i + 1) + ".h5")   

    y_pred = saved_model.predict_classes(X_test)    

    # CNN metrics
    accuracyScore, precisionScoreBinary, recallScoreBinary, f1ScoreBinary = util.get_testing_metric(y_test, y_pred)

    now = time.time()
    diff = now - then
    print(diff)
    print(accuracyScore)

    # save testing data
    f = open(resultsPath + testing_name + ".csv", 'a')
    f.write(str(i + 1)
            + ',' + str(hyperparameters['epochs'])
            + ',' + str(hyperparameters['batch_size'])
            + ',' + str(hyperparameters['optimizer'])
            + ',' + str(hyperparameters['learning_rate'])
            + ',' + str(hyperparameters['output_dim'])
            + ',' + str(hyperparameters['conv_activation_func1'])
            + ',' + str(accuracyScore)
            + ',' + str(precisionScoreBinary)
            + ',' + str(recallScoreBinary)
            + ',' + str(f1ScoreBinary)
            + ',' + str(diff) + '\n')
    f.close()

1


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


12.11611533164978
0.6111822449850619
2


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


8.420685529708862
0.6111822449850619
3
48.46009063720703
0.7285531370038413
4
47.44665265083313
0.7622705932565087
5
9.918481826782227
0.6120358514724712
6


...

/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


19.073810815811157
0.6111822449850619
147


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


15.552733182907104
0.6111822449850619
148
29.122811555862427
0.813486982501067
149
35.814653635025024
0.8113529662825437
150
15.891812801361084
0.6111822449850619


/lab/dbms/fatyanosa/.userprogram/anaconda3/envs/GA-CNN/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
%%javascript
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>